# Trasformazioni

prepariamo il Dataset per le analisi successive: Clustering; Classification; Association.

1) Clustering:

abbiamo bisogno dei soli attributi numerici standardizzati. Inoltre il dataset è troppo grosso, quindi operiamo un Sample su di esso condizionato a raggruppamenti categorici

2) Classification:


3) Association:

** Import DataSet e Librerie Data Understanding **

In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats.stats import pearsonr

df = pd.read_csv('Training_DU.csv')
df_Test = pd.read_csv('Test_DU.csv')

In [2]:
df_Clustering = df.copy()
df_Classification = df.copy()
df_Association = df.copy()

In [3]:
df_Test_Classification = df_Test.copy()
df_Test_Association = df_Test.copy()

In [4]:
del df

In [5]:
del df_Test

## 1) Tasformazioni per il Clustering

In [6]:
df_Clustering.count()

PurchDate                            50906
Model                                50906
SubModel                             50906
VehYear                              50906
RefId                                50906
IsBadBuy                             50906
Auction                              50906
VehicleAge                           50906
Make                                 50906
Engine                               33078
Trim                                 50906
Transmission                         50906
WheelTypeID                          50906
VehOdo                               50906
Nationality                          50906
Size                                 50906
TopThreeAmericanName                 50906
MMRAcquisitionAuctionAveragePrice    50906
BYRNO                                50906
VNZIP1                               50906
VNST                                 50906
VehBCost                             50906
IsOnlineSale                         50906
WarrantyCos

In [5]:
df_Clustering['Model'].nunique()

261

In [6]:
df_Clustering['SubModel'].nunique()

38

In [7]:
df_Clustering['VehYear'].nunique()

10

** Trasformazioni variabili categoriche in numeriche **

trasformiamo WheelTypeID,Transmission,Nationality in variabili numeriche per usarle anche nella Cluster Analysis

In [8]:
#1 TRANSMISSION
#Clustering_df['Transmission'].unique()

In [9]:
#transm = sorted(Clustering_df['Transmission'].unique())
#transm_mapping = dict(zip(transm, range(0, len(transm) + 1)))
#transm_mapping

In [10]:
#Clustering_df['TransmissionID'] = Clustering_df['Transmission'].map(transm_mapping).astype(int)
#Clustering_df[['Transmission', 'TransmissionID']].head()

In [11]:
#2 NATIONALITY
#Clustering_df['Nationality'].unique()

In [12]:
#nation = sorted(Clustering_df['Nationality'].unique())
#nation_mapping = dict(zip(nation, range(0, len(nation) + 1)))
#nation_mapping

In [13]:
#Clustering_df['NationalityID'] = Clustering_df['Nationality'].map(nation_mapping).astype(int)
#Clustering_df[['Nationality', 'NationalityID']].head()

** nuove variabili **

In [14]:
df_Clustering['CostOverOdo'] = df_Clustering['VehBCost'] / df_Clustering['VehOdo']

In [15]:
df_Clustering['TotalPrice'] = df_Clustering['VehBCost'] + df_Clustering['WarrantyCost']

In [16]:
df_Clustering.head()

,PurchDate,Model,SubModel,VehYear,RefId,IsBadBuy,Auction,VehicleAge,Make,Engine,...,MMRAcquisitionAuctionAveragePrice,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,Doors,CostOverOdo,TotalPrice
0,2009-01-04,1500 RAM,QUAD CAB,2002,62472,0,ADESA,7,DODGE,5.9L,...,5872.0,20234,85226,AZ,7940.0,0,1125,NaN,0.127634,9065.0
1,2009-01-04,1500 RAM,QUAD CAB,2003,16342,0,MANHEIM,6,DODGE,5.7L,...,7648.0,19619,32824,FL,9400.0,0,1220,NaN,0.117610,10620.0
2,2009-01-04,1500 RAM,QUAD CAB,2005,16344,0,MANHEIM,4,DODGE,5.7L,...,10306.0,19619,32824,FL,10000.0,0,920,NaN,0.126894,10920.0
3,2009-01-04,300,SEDAN,2006,62463,0,ADESA,3,CHRYSLER,2.7L MPI,...,9459.0,20234,85226,AZ,9505.0,0,1389,4D,0.132707,10894.0
4,2009-01-04,300,SEDAN,2006,20574,0,MANHEIM,3,CHRYSLER,2.7L MPI,...,9459.0,20833,75236,TX,10050.0,0,1215,4D,0.156352,11265.0


In [17]:
df_Clustering[df_Clustering.columns.difference(['RefId', 'TopThreeAmericanName', 'IsBadBuy', 'PurchDate', 'Auction', 'Make', 'Engine', 'Trim',\
                                                'Size','BYRNO', 'VNZIP1', 'VNST', 'IsOnlineSale', 'Model', 'SubModel', 'Doors', 'Nationality',\
                                                'Transmission','Model','SubModel','MMRAcquisitionAuctionAveragePrice','MMRAcquisitionRetailAveragePrice',\
                                               'MMRCurrentAuctionAveragePrice','MMRCurrentRetailAveragePrice','MMRCurrentRetailAveragePrice',\
                                               'VehYear', 'VehicleAge','WheelTypeID'])].corr() 

,CostOverOdo,TotalPrice,VehBCost,VehOdo,WarrantyCost
CostOverOdo,1.000000,0.543756,0.654875,-0.618816,-0.249602
TotalPrice,0.543756,1.000000,0.943251,0.070288,0.292776
VehBCost,0.654875,0.943251,1.000000,-0.068337,-0.041367
VehOdo,-0.618816,0.070288,-0.068337,1.000000,0.408248
WarrantyCost,-0.249602,0.292776,-0.041367,0.408248,1.000000


In [18]:
del df_Clustering['TotalPrice']

** Operiamo un Sample sul DataSet condizionato a raggruppamenti categorici **

In [19]:
def Sample_df(x):
    return x.sample(frac=1, replace=True, random_state=1)

In [20]:
df_Clustering = df_Clustering.groupby([df_Clustering['Model'], df_Clustering['SubModel'],
                                       df_Clustering['VehYear']]).apply(Sample_df)

In [21]:
df_Clustering.count()

PurchDate                            50906
Model                                50906
SubModel                             50906
VehYear                              50906
RefId                                50906
IsBadBuy                             50906
Auction                              50906
VehicleAge                           50906
Make                                 50906
Engine                               33676
Trim                                 50906
Transmission                         50906
WheelTypeID                          50906
VehOdo                               50906
Nationality                          50906
Size                                 50906
TopThreeAmericanName                 50906
MMRAcquisitionAuctionAveragePrice    50906
BYRNO                                50906
VNZIP1                               50906
VNST                                 50906
VehBCost                             50906
IsOnlineSale                         50906
WarrantyCos

In [22]:
df_Clustering['Model'].nunique()

261

In [23]:
df_Clustering['SubModel'].nunique()

38

In [24]:
df_Clustering['VehYear'].nunique()

10

Salviamo il Dataframe attuale in una variabile per esportarlo in un file csv: <br>
ci servirà per ricostruire il dataset finale delle analisi di Clustering (con tutti i valori originali del sample in esame e le label dei cluster in fondo).

In [25]:
df_Clustering_forResultsAnalysis = df_Clustering.copy()
df_Clustering_forResultsAnalysis

PurchDate     Model  SubModel  VehYear  \
Model    SubModel VehYear                                                  
1500 RAM EXT CAB  2001    13133  2009-07-28  1500 RAM   EXT CAB     2001   
                          13133  2009-07-28  1500 RAM   EXT CAB     2001   
         MEGA CAB 2006    3440   2009-02-25  1500 RAM  MEGA CAB     2006   
                          556    2009-01-12  1500 RAM  MEGA CAB     2006   
                          556    2009-01-12  1500 RAM  MEGA CAB     2006   
         QUAD CAB 2002    49681  2010-12-08  1500 RAM  QUAD CAB     2002   
                          13562  2009-07-31  1500 RAM  QUAD CAB     2002   
                          9794   2009-05-28  1500 RAM  QUAD CAB     2002   
                          11263  2009-06-25  1500 RAM  QUAD CAB     2002   
                          12015  2009-07-10  1500 RAM  QUAD CAB     2002   
                          5304   2009-03-25  1500 RAM  QUAD CAB     2002   
                          22995  2009-12-08  1500 RAM  QUAD CAB     2002   
                          0      2009-01-04  1500 RAM  QUAD CAB     2002   
                          24572  2010-01-07  1500 RAM  QUAD CAB     2002   
                          996    2009-01-20  1500 RAM  QUAD CAB     2002   
                          13562  2009-07-31  1500 RAM  QUAD CAB     2002   
                          7900   2009-04-30  1500 RAM  QUAD CAB     2002   
                          7772   2009-04-29  1500 RAM  QUAD CAB     2002   
                          32885  2010-04-30  1500 RAM  QUAD CAB     2002   
                          28815  2010-03-03  1500 RAM  QUAD CAB     2002   
                          49681  2010-12-08  1500 RAM  QUAD CAB     2002   
                          26945  2010-02-07  1500 RAM  QUAD CAB     2002   
                          28815  2010-03-03  1500 RAM  QUAD CAB     2002   
                          12015  2009-07-10  1500 RAM  QUAD CAB     2002   
                          35963  2010-06-16  1500 RAM  QUAD CAB     2002   
                          36859  2010-06-29  1500 RAM  QUAD CAB     2002   
                          20400  2009-10-28  1500 RAM  QUAD CAB     2002   
                          2666   2009-02-17  1500 RAM  QUAD CAB     2002   
                          30346  2010-03-25  1500 RAM  QUAD CAB     2002   
                          30346  2010-03-25  1500 RAM  QUAD CAB     2002   
...                                     ...       ...       ...      ...   
YUKON XL UTILITY  2003    790    2009-01-13  YUKON XL   UTILITY     2003   
                          16959  2009-09-16  YUKON XL   UTILITY     2003   
                          5113   2009-03-19  YUKON XL   UTILITY     2003   
                          18106  2009-09-30  YUKON XL   UTILITY     2003   
                          3676   2009-02-25  YUKON XL   UTILITY     2003   
                          40000  2010-08-06  YUKON XL   UTILITY     2003   
                          41195  2010-08-24  YUKON XL   UTILITY     2003   
                          3031   2009-02-18  YUKON XL   UTILITY     2003   
                          40000  2010-08-06  YUKON XL   UTILITY     2003   
                          41195  2010-08-24  YUKON XL   UTILITY     2003   
                          12391  2009-07-14  YUKON XL   UTILITY     2003   
                  2004    39392  2010-07-28  YUKON XL   UTILITY     2004   
                          23912  2009-12-18  YUKON XL   UTILITY     2004   
                          26144  2010-01-26  YUKON XL   UTILITY     2004   
                          791    2009-01-13  YUKON XL   UTILITY     2004   
                          5572   2009-03-30  YUKON XL   UTILITY     2004   
                          23912  2009-12-18  YUKON XL   UTILITY     2004   
                  2005    1339   2009-01-22  YUKON XL   UTILITY     2005   
ZEPHYR   SEDAN    2006    27892  2010-02-18    ZEPHYR     SEDAN     2006   
                          48665  2010-11-23    ZEPHYR     SEDAN     2006   
      

** Eliminazione attributi categorici **

In [26]:
df_Clustering.dtypes

PurchDate                             object
Model                                 object
SubModel                              object
VehYear                                int64
RefId                                  int64
IsBadBuy                               int64
Auction                               object
VehicleAge                             int64
Make                                  object
Engine                                object
Trim                                  object
Transmission                          object
WheelTypeID                          float64
VehOdo                                 int64
Nationality                           object
Size                                  object
TopThreeAmericanName                  object
MMRAcquisitionAuctionAveragePrice    float64
BYRNO                                  int64
VNZIP1                                 int64
VNST                                  object
VehBCost                             float64
IsOnlineSa

In [27]:
col_toDrop = ['VehYear', 'RefId', 'TopThreeAmericanName', 'IsBadBuy', 'PurchDate', 'Auction', 'Make', 'Engine',
              'WheelTypeID', 'Trim', 'Size', 'BYRNO', 'VNZIP1', 'VNST', 'IsOnlineSale', 'Model', 'SubModel', 'Doors',
              'Nationality', 'Transmission']

df_Clustering = df_Clustering.drop(col_toDrop, axis=1)
df_Clustering

VehicleAge  VehOdo  \
Model    SubModel VehYear                             
1500 RAM EXT CAB  2001    13133           8   55117   
                          13133           8   55117   
         MEGA CAB 2006    3440            3   72728   
                          556             3   80650   
                          556             3   80650   
         QUAD CAB 2002    49681           8   87788   
                          13562           7   58350   
                          9794            7   87034   
                          11263           7   74233   
                          12015           7   79561   
                          5304            7   88290   
                          22995           7   51225   
                          0               7   62209   
                          24572           8   58260   
                          996             7   61873   
                          13562           7   58350   
                          7900            7   94808   
                          7772            7   74369   
                          32885           8   78282   
                          28815           8   81070   
                          49681           8   87788   
                          26945           8   86511   
                          28815           8   81070   
                          12015           7   79561   
                          35963           8   72960   
                          36859           8   72101   
                          20400           7   83212   
                          2666            7   77662   
                          30346           8   86431   
                          30346           8   86431   
...                                     ...     ...   
YUKON XL UTILITY  2003    790             6   65490   
                          16959           6   76626   
                          5113            6   77952   
                          18106           6   83054   
                          3676            6   89214   
                          40000           7   92910   
                          41195           7   82652   
                          3031            6   79500   
                          40000           7   92910   
                          41195           7   82652   
                          12391           6   75429   
                  2004    39392           6   84672   
                          23912           5   62269   
                          26144           6   77985   
                          791             5   57617   
                          5572            5   80178   
                          23912           5   62269   
                  2005    1339            4   71990   
ZEPHYR   SEDAN    2006    27892           4   82286   
                          48665           4   80855   
                          44262           4   82020   
                          44933           4   73564   
                          48665           4   80855   
                          27892           4   82286   
                          4128            3   68976   
                          4128            3   68976   
                          6852            3   68272   
                          43333           4   72138   
                          38256           4   70797   
                          44933           4   73564   

                                 MMRAcquisitionAuctionAveragePrice  VehBCost  \
Model    SubModel VehYear                                                      
1500 RAM EXT CAB  2001    13133                             3192.0    4315.0   
                          13133                             3192.0    4315.0   
         MEGA CAB 2006    3440                             12423.0   10835.0   
                          556                              13271.0   11515.0   
                          556                              13271.0   11515.0   
       

** Scaling **

In [28]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()

In [29]:
scaler.fit(df_Clustering)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [30]:
col_toScale = df_Clustering.columns

df_Clustering = scaler.transform(df_Clustering)
df_Clustering = pd.DataFrame(df_Clustering, columns = col_toScale)
df_Clustering

,VehicleAge,VehOdo,MMRAcquisitionAuctionAveragePrice,VehBCost,WarrantyCost,CostOverOdo
0,2.192212,-1.100717,-1.225996,-1.325668,-0.350554,-0.477847
1,2.192212,-1.100717,-1.225996,-1.325668,-0.350554,-0.477847
2,-0.675803,0.102976,2.611267,2.338646,-0.568878,1.138453
3,-0.675803,0.644436,2.963774,2.720813,-0.466233,0.996642
4,-0.675803,0.644436,2.963774,2.720813,-0.466233,0.996642
5,2.192212,1.132310,-0.355535,1.369176,0.022553,0.104837
6,1.618609,-0.879745,-0.217940,-0.122964,-0.350554,0.261520
7,1.618609,1.080775,-1.098378,1.298925,0.022553,0.092554
8,1.618609,0.205841,-1.098378,0.801545,-0.080092,0.227004
9,1.618609,0.570004,-1.079672,0.624511,-0.080092,-0.030593


In [31]:
# per eventualmente riportare il dataset ai valori originali
#pd.DataFrame(scaler.inverse_transform(df_Clustering[col_toScale].head().values), columns = col_toScale)

## 2) Tasformazioni per la Association

In [7]:
df_Association.isnull().sum()

PurchDate                                0
Model                                    0
SubModel                                 0
VehYear                                  0
RefId                                    0
IsBadBuy                                 0
Auction                                  0
VehicleAge                               0
Make                                     0
Engine                               17828
Trim                                     0
Transmission                             0
WheelTypeID                              0
VehOdo                                   0
Nationality                              0
Size                                     0
TopThreeAmericanName                     0
MMRAcquisitionAuctionAveragePrice        0
BYRNO                                    0
VNZIP1                                   0
VNST                                     0
VehBCost                                 0
IsOnlineSale                             0
WarrantyCos

** Modifiche degli attributi **

- 'PurchDate'

di questo attributo teniamo solo i valori di mese e anno (non il giorno) in maniera da ridurre il numero di categorie di valori.

In [8]:
df_Association["PurchDate"].nunique()

516

In [9]:
df_Association['PurchYearMonth'] = pd.to_datetime(df_Association['PurchDate']).dt.to_period('M')
df_Association['PurchYearMonth']

0       2009-01
1       2009-01
2       2009-01
3       2009-01
4       2009-01
5       2009-01
6       2009-01
7       2009-01
8       2009-01
9       2009-01
10      2009-01
11      2009-01
12      2009-01
13      2009-01
14      2009-01
15      2009-01
16      2009-01
17      2009-01
18      2009-01
19      2009-01
20      2009-01
21      2009-01
22      2009-01
23      2009-01
24      2009-01
25      2009-01
26      2009-01
27      2009-01
28      2009-01
29      2009-01
          ...  
50876   2010-12
50877   2010-12
50878   2010-12
50879   2010-12
50880   2010-12
50881   2010-12
50882   2010-12
50883   2010-12
50884   2010-12
50885   2010-12
50886   2010-12
50887   2010-12
50888   2010-12
50889   2010-12
50890   2010-12
50891   2010-12
50892   2010-12
50893   2010-12
50894   2010-12
50895   2010-12
50896   2010-12
50897   2010-12
50898   2010-12
50899   2010-12
50900   2010-12
50901   2010-12
50902   2010-12
50903   2010-12
50904   2010-12
50905   2010-12
Name: PurchYearMonth, Le

In [10]:
df_Association["PurchYearMonth"].nunique()

24

In [11]:
del df_Association['PurchDate']

- 'RefId'

In [12]:
del df_Association['RefId']

** Binning delle Variabili Numeriche **

In [13]:
df_Association.nunique()

Model                                  261
SubModel                                38
VehYear                                 10
IsBadBuy                                 2
Auction                                  3
VehicleAge                              10
Make                                    32
Engine                                 127
Trim                                   129
Transmission                             2
WheelTypeID                              3
VehOdo                               32709
Nationality                              2
Size                                    12
TopThreeAmericanName                     4
MMRAcquisitionAuctionAveragePrice     9703
BYRNO                                   72
VNZIP1                                 152
VNST                                    37
VehBCost                              1982
IsOnlineSale                             2
WarrantyCost                           276
Doors                                    4
PurchYearMo

In [14]:
df_Association.dtypes

Model                                 object
SubModel                              object
VehYear                                int64
IsBadBuy                               int64
Auction                               object
VehicleAge                             int64
Make                                  object
Engine                                object
Trim                                  object
Transmission                          object
WheelTypeID                          float64
VehOdo                                 int64
Nationality                           object
Size                                  object
TopThreeAmericanName                  object
MMRAcquisitionAuctionAveragePrice    float64
BYRNO                                  int64
VNZIP1                                 int64
VNST                                  object
VehBCost                             float64
IsOnlineSale                           int64
WarrantyCost                           int64
Doors     

In [15]:
col_toBin = df_Association.select_dtypes(['float64', 'int64']).columns.difference(['VNZIP1', 'IsBadBuy', 'IsOnlineSale',
                                                                                   'BYRNO', 'VehicleAge', 'WheelTypeID'])
col_toBin

Index(['MMRAcquisitionAuctionAveragePrice', 'VehBCost', 'VehOdo', 'VehYear',
       'WarrantyCost'],
      dtype='object')

In [16]:
Bins = math.ceil(math.log(df_Association.shape[0],2))+1
Bins

17

In [17]:
for col in col_toBin:
    if df_Association[col].nunique() > Bins:
        df_Association[col + 'Bin'] = pd.cut(df_Association[col].astype(int), Bins, right=False)
        del df_Association[col]

df_Association.nunique()

Model                                   261
SubModel                                 38
VehYear                                  10
IsBadBuy                                  2
Auction                                   3
VehicleAge                               10
Make                                     32
Engine                                  127
Trim                                    129
Transmission                              2
WheelTypeID                               3
Nationality                               2
Size                                     12
TopThreeAmericanName                      4
BYRNO                                    72
VNZIP1                                  152
VNST                                     37
IsOnlineSale                              2
Doors                                     4
PurchYearMonth                           24
MMRAcquisitionAuctionAveragePriceBin     15
VehBCostBin                              14
VehOdoBin                       

In [18]:
df_Association

,Model,SubModel,VehYear,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,Transmission,...,BYRNO,VNZIP1,VNST,IsOnlineSale,Doors,PurchYearMonth,MMRAcquisitionAuctionAveragePriceBin,VehBCostBin,VehOdoBin,WarrantyCostBin
0,1500 RAM,QUAD CAB,2002,0,ADESA,7,DODGE,5.9L,ST,AUTO,...,20234,85226,AZ,0,NaN,2009-01,"[4982.588, 7031.882)","[6439.353, 8585.471)","[57296.941, 63788.059)","[875.882, 1289.765)"
1,1500 RAM,QUAD CAB,2003,0,MANHEIM,6,DODGE,5.7L,LAR,AUTO,...,19619,32824,FL,0,NaN,2009-01,"[7031.882, 9081.176)","[8585.471, 10731.588)","[76770.294, 83261.412)","[875.882, 1289.765)"
2,1500 RAM,QUAD CAB,2005,0,MANHEIM,4,DODGE,5.7L,SLT,AUTO,...,19619,32824,FL,0,NaN,2009-01,"[9081.176, 11130.471)","[8585.471, 10731.588)","[76770.294, 83261.412)","[875.882, 1289.765)"
3,300,SEDAN,2006,0,ADESA,3,CHRYSLER,2.7L MPI,BAS,AUTO,...,20234,85226,AZ,0,4D,2009-01,"[9081.176, 11130.471)","[8585.471, 10731.588)","[70279.176, 76770.294)","[1289.765, 1703.647)"
4,300,SEDAN,2006,0,MANHEIM,3,CHRYSLER,2.7L MPI,BAS,AUTO,...,20833,75236,TX,0,4D,2009-01,"[9081.176, 11130.471)","[8585.471, 10731.588)","[63788.059, 70279.176)","[875.882, 1289.765)"
5,300,SEDAN,2006,0,MANHEIM,3,CHRYSLER,2.7L MPI,BAS,AUTO,...,20928,32824,FL,0,4D,2009-01,"[9081.176, 11130.471)","[8585.471, 10731.588)","[50805.824, 57296.941)","[875.882, 1289.765)"
6,ACCENT,SEDAN,2004,0,OTHER,5,HYUNDAI,1.6L MPI,GL,AUTO,...,835,85009,AZ,0,4D,2009-01,"[884.0, 2933.294)","[2147.118, 4293.235)","[70279.176, 76770.294)","[875.882, 1289.765)"
7,ACCENT,SEDAN,2005,1,OTHER,4,HYUNDAI,1.6L MPI,GLS,AUTO,...,835,85009,AZ,0,4D,2009-01,"[2933.294, 4982.588)","[4293.235, 6439.353)","[50805.824, 57296.941)","[462.0, 875.882)"
8,ACCORD,COUPE,2005,0,MANHEIM,4,HONDA,4C 2.4L,SE,AUTO,...,20392,75236,TX,0,2D,2009-01,"[7031.882, 9081.176)","[6439.353, 8585.471)","[70279.176, 76770.294)","[462.0, 875.882)"
9,AERIO,SEDAN,2003,1,OTHER,6,SUZUKI,2.0L EFI,S,AUTO,...,835,85009,AZ,0,4D,2009-01,"[884.0, 2933.294)","[4293.235, 6439.353)","[70279.176, 76770.294)","[875.882, 1289.765)"


** Append dell'Intestazione delle Colonne **

In [19]:
for col in df_Association.columns:
    df_Association[col] = df_Association[col].astype(str) + '_' + str(col)

In [20]:
df_Association

,Model,SubModel,VehYear,IsBadBuy,Auction,VehicleAge,Make,Engine,Trim,Transmission,...,BYRNO,VNZIP1,VNST,IsOnlineSale,Doors,PurchYearMonth,MMRAcquisitionAuctionAveragePriceBin,VehBCostBin,VehOdoBin,WarrantyCostBin
0,1500 RAM_Model,QUAD CAB_SubModel,2002_VehYear,0_IsBadBuy,ADESA_Auction,7_VehicleAge,DODGE_Make,5.9L_Engine,ST_Trim,AUTO_Transmission,...,20234_BYRNO,85226_VNZIP1,AZ_VNST,0_IsOnlineSale,nan_Doors,2009-01_PurchYearMonth,"[4982.588, 7031.882)_MMRAcquisitionAuctionAver...","[6439.353, 8585.471)_VehBCostBin","[57296.941, 63788.059)_VehOdoBin","[875.882, 1289.765)_WarrantyCostBin"
1,1500 RAM_Model,QUAD CAB_SubModel,2003_VehYear,0_IsBadBuy,MANHEIM_Auction,6_VehicleAge,DODGE_Make,5.7L_Engine,LAR_Trim,AUTO_Transmission,...,19619_BYRNO,32824_VNZIP1,FL_VNST,0_IsOnlineSale,nan_Doors,2009-01_PurchYearMonth,"[7031.882, 9081.176)_MMRAcquisitionAuctionAver...","[8585.471, 10731.588)_VehBCostBin","[76770.294, 83261.412)_VehOdoBin","[875.882, 1289.765)_WarrantyCostBin"
2,1500 RAM_Model,QUAD CAB_SubModel,2005_VehYear,0_IsBadBuy,MANHEIM_Auction,4_VehicleAge,DODGE_Make,5.7L_Engine,SLT_Trim,AUTO_Transmission,...,19619_BYRNO,32824_VNZIP1,FL_VNST,0_IsOnlineSale,nan_Doors,2009-01_PurchYearMonth,"[9081.176, 11130.471)_MMRAcquisitionAuctionAve...","[8585.471, 10731.588)_VehBCostBin","[76770.294, 83261.412)_VehOdoBin","[875.882, 1289.765)_WarrantyCostBin"
3,300_Model,SEDAN_SubModel,2006_VehYear,0_IsBadBuy,ADESA_Auction,3_VehicleAge,CHRYSLER_Make,2.7L MPI_Engine,BAS_Trim,AUTO_Transmission,...,20234_BYRNO,85226_VNZIP1,AZ_VNST,0_IsOnlineSale,4D_Doors,2009-01_PurchYearMonth,"[9081.176, 11130.471)_MMRAcquisitionAuctionAve...","[8585.471, 10731.588)_VehBCostBin","[70279.176, 76770.294)_VehOdoBin","[1289.765, 1703.647)_WarrantyCostBin"
4,300_Model,SEDAN_SubModel,2006_VehYear,0_IsBadBuy,MANHEIM_Auction,3_VehicleAge,CHRYSLER_Make,2.7L MPI_Engine,BAS_Trim,AUTO_Transmission,...,20833_BYRNO,75236_VNZIP1,TX_VNST,0_IsOnlineSale,4D_Doors,2009-01_PurchYearMonth,"[9081.176, 11130.471)_MMRAcquisitionAuctionAve...","[8585.471, 10731.588)_VehBCostBin","[63788.059, 70279.176)_VehOdoBin","[875.882, 1289.765)_WarrantyCostBin"
5,300_Model,SEDAN_SubModel,2006_VehYear,0_IsBadBuy,MANHEIM_Auction,3_VehicleAge,CHRYSLER_Make,2.7L MPI_Engine,BAS_Trim,AUTO_Transmission,...,20928_BYRNO,32824_VNZIP1,FL_VNST,0_IsOnlineSale,4D_Doors,2009-01_PurchYearMonth,"[9081.176, 11130.471)_MMRAcquisitionAuctionAve...","[8585.471, 10731.588)_VehBCostBin","[50805.824, 57296.941)_VehOdoBin","[875.882, 1289.765)_WarrantyCostBin"
6,ACCENT_Model,SEDAN_SubModel,2004_VehYear,0_IsBadBuy,OTHER_Auction,5_VehicleAge,HYUNDAI_Make,1.6L MPI_Engine,GL_Trim,AUTO_Transmission,...,835_BYRNO,85009_VNZIP1,AZ_VNST,0_IsOnlineSale,4D_Doors,2009-01_PurchYearMonth,"[884.0, 2933.294)_MMRAcquisitionAuctionAverage...","[2147.118, 4293.235)_VehBCostBin","[70279.176, 76770.294)_VehOdoBin","[875.882, 1289.765)_WarrantyCostBin"
7,ACCENT_Model,SEDAN_SubModel,2005_VehYear,1_IsBadBuy,OTHER_Auction,4_VehicleAge,HYUNDAI_Make,1.6L MPI_Engine,GLS_Trim,AUTO_Transmission,...,835_BYRNO,85009_VNZIP1,AZ_VNST,0_IsOnlineSale,4D_Doors,2009-01_PurchYearMonth,"[2933.294, 4982.588)_MMRAcquisitionAuctionAver...","[4293.235, 6439.353)_VehBCostBin","[50805.824, 57296.941)_VehOdoBin","[462.0, 875.882)_WarrantyCostBin"
8,ACCORD_Model,COUPE_SubModel,2005_VehYear,0_IsBadBuy,MANHEIM_Auction,4_VehicleAge,HONDA_Make,4C 2.4L_Engine,SE_Trim,AUTO_Transmission,...,20392_BYRNO,75236_VNZIP1,TX_VNST,0_IsOnlineSale,2D_Doors,2009-01_PurchYearMonth,"[7031.882, 9081.176)_MMRAcquisitionAuctionAver...","[6439.353, 8585.471)_VehBCostBin","[70279.176, 76770.294)_VehOdoBin","[462.0, 875.882)_WarrantyCostBin"
9,AERIO_Model,SEDAN_SubModel,2003_VehYear,1_IsBadBuy,OTHER_Auction,6_VehicleAge,SUZUKI_Make,2.0L EFI_Engine,S_Trim,AUTO_Transmission,...,835_BYRNO,85009_VNZIP1,AZ_VNST,0_IsOnlineSale,4D_Doors,2009-01_PurchYearMonth,"[884.0, 2933.294)_MMRAcquisitionAuctionAverage...","[4293.235, 6439.353)_VehBCostBin","[70279.176, 76770.294)_VehOdoBin","[875.882, 1289.765)_Warran

In [21]:
df_Association.nunique()

Model                                   261
SubModel                                 38
VehYear                                  10
IsBadBuy                                  2
Auction                                   3
VehicleAge                               10
Make                                     32
Engine                                  128
Trim                                    129
Transmission                              2
WheelTypeID                               3
Nationality                               2
Size                                     12
TopThreeAmericanName                      4
BYRNO                                    72
VNZIP1                                  152
VNST                                     37
IsOnlineSale                              2
Doors                                     5
PurchYearMonth                           24
MMRAcquisitionAuctionAveragePriceBin     15
VehBCostBin                              14
VehOdoBin                       

In [22]:
df_Association.isnull().sum()

Model                                   0
SubModel                                0
VehYear                                 0
IsBadBuy                                0
Auction                                 0
VehicleAge                              0
Make                                    0
Engine                                  0
Trim                                    0
Transmission                            0
WheelTypeID                             0
Nationality                             0
Size                                    0
TopThreeAmericanName                    0
BYRNO                                   0
VNZIP1                                  0
VNST                                    0
IsOnlineSale                            0
Doors                                   0
PurchYearMonth                          0
MMRAcquisitionAuctionAveragePriceBin    0
VehBCostBin                             0
VehOdoBin                               0
WarrantyCostBin                   

*** Stesso procedimento per il Test set ***

In [23]:
df_Test_Association.isnull().sum()

RefId                                   0
IsBadBuy                                0
PurchDate                               0
Auction                                 0
VehYear                                 0
VehicleAge                              0
Make                                    0
Engine                               4883
Trim                                  430
Transmission                            1
WheelTypeID                           596
VehOdo                                  0
Nationality                             1
Size                                    1
TopThreeAmericanName                    1
MMRAcquisitionAuctionAveragePrice       5
MMRAcquisitionAuctionCleanPrice         5
MMRAcquisitionRetailAveragePrice        5
MMRAcquisitonRetailCleanPrice           5
MMRCurrentAuctionAveragePrice          70
MMRCurrentAuctionCleanPrice            70
MMRCurrentRetailAveragePrice           70
MMRCurrentRetailCleanPrice             70
BYRNO                             

** Modifiche degli attributi **

- 'PurchDate'

di questo attributo teniamo solo i valori di mese e anno (non il giorno) in maniera da ridurre il numero di categorie di valori.

In [24]:
df_Test_Association["PurchDate"].nunique()

509

In [25]:
df_Test_Association['PurchYearMonth'] = pd.to_datetime(df_Test_Association['PurchDate']).dt.to_period('M')
df_Test_Association['PurchYearMonth']

0       2010-10
1       2010-01
2       2009-09
3       2010-01
4       2009-03
5       2010-12
6       2010-01
7       2009-07
8       2010-06
9       2009-03
10      2010-08
11      2009-04
12      2010-08
13      2010-12
14      2010-02
15      2010-12
16      2010-03
17      2009-03
18      2010-06
19      2009-02
20      2009-10
21      2009-07
22      2010-04
23      2010-09
24      2010-07
25      2010-01
26      2009-06
27      2009-09
28      2010-05
29      2009-03
          ...  
14567   2009-08
14568   2009-07
14569   2010-03
14570   2009-03
14571   2009-09
14572   2010-03
14573   2010-04
14574   2010-08
14575   2009-07
14576   2010-11
14577   2010-07
14578   2009-12
14579   2009-12
14580   2009-09
14581   2009-09
14582   2009-09
14583   2010-02
14584   2009-08
14585   2010-05
14586   2010-07
14587   2010-12
14588   2010-08
14589   2010-08
14590   2009-03
14591   2010-02
14592   2009-09
14593   2010-08
14594   2009-07
14595   2009-06
14596   2009-11
Name: PurchYearMonth, Le

In [26]:
df_Test_Association["PurchYearMonth"].nunique()

24

In [27]:
del df_Test_Association['PurchDate']

- 'RefId'

In [28]:
del df_Test_Association['RefId']

- MMR

In [29]:
del df_Test_Association['MMRAcquisitionAuctionCleanPrice']
del df_Test_Association['MMRAcquisitionRetailAveragePrice']
del df_Test_Association['MMRAcquisitonRetailCleanPrice']
del df_Test_Association['MMRCurrentAuctionAveragePrice']
del df_Test_Association['MMRCurrentAuctionCleanPrice']
del df_Test_Association['MMRCurrentRetailAveragePrice']
del df_Test_Association['MMRCurrentRetailCleanPrice']

In [30]:
df_Test_Association = df_Test_Association.dropna(subset=['MMRAcquisitionAuctionAveragePrice'])

In [31]:
df_Test_Association['MMRAcquisitionAuctionAveragePrice'].isnull().sum()

0

** Binning delle Variabili Numeriche **

In [32]:
df_Test_Association.nunique()

IsBadBuy                                 2
Auction                                  3
VehYear                                  9
VehicleAge                              10
Make                                    31
Engine                                 110
Trim                                   115
Transmission                             2
WheelTypeID                              4
VehOdo                               12737
Nationality                              2
Size                                    12
TopThreeAmericanName                     4
MMRAcquisitionAuctionAveragePrice     6628
BYRNO                                   70
VNZIP1                                 142
VNST                                    37
VehBCost                              1667
IsOnlineSale                             2
WarrantyCost                           246
Model                                  223
Doors                                    4
SubModel                                31
PurchYearMo

In [33]:
df_Test_Association.dtypes

IsBadBuy                               int64
Auction                               object
VehYear                                int64
VehicleAge                             int64
Make                                  object
Engine                                object
Trim                                  object
Transmission                          object
WheelTypeID                          float64
VehOdo                                 int64
Nationality                           object
Size                                  object
TopThreeAmericanName                  object
MMRAcquisitionAuctionAveragePrice    float64
BYRNO                                  int64
VNZIP1                                 int64
VNST                                  object
VehBCost                             float64
IsOnlineSale                           int64
WarrantyCost                           int64
Model                                 object
Doors                                 object
SubModel  

In [34]:
col_toBin = df_Test_Association.select_dtypes(['float64', 'int64']).columns.difference(['VNZIP1', 'IsBadBuy', 'IsOnlineSale',
                                                                                        'BYRNO', 'VehicleAge', 'WheelTypeID',
                                                                                       'VehYear'])
col_toBin

Index(['MMRAcquisitionAuctionAveragePrice', 'VehBCost', 'VehOdo',
       'WarrantyCost'],
      dtype='object')

In [35]:
Bins = math.ceil(math.log(df_Test_Association.shape[0],2))+1
Bins

15

In [36]:
for col in col_toBin:
    if df_Test_Association[col].nunique() > Bins:
        df_Test_Association[col + 'Bin'] = pd.cut(df_Test_Association[col].astype(int), Bins, right=False)
        del df_Test_Association[col]

df_Test_Association.nunique()

IsBadBuy                                  2
Auction                                   3
VehYear                                   9
VehicleAge                               10
Make                                     31
Engine                                  110
Trim                                    115
Transmission                              2
WheelTypeID                               4
Nationality                               2
Size                                     12
TopThreeAmericanName                      4
BYRNO                                    70
VNZIP1                                  142
VNST                                     37
IsOnlineSale                              2
Model                                   223
Doors                                     4
SubModel                                 31
PurchYearMonth                           24
MMRAcquisitionAuctionAveragePriceBin     11
VehBCostBin                               8
VehOdoBin                       

In [37]:
df_Test_Association

,IsBadBuy,Auction,VehYear,VehicleAge,Make,Engine,Trim,Transmission,WheelTypeID,Nationality,...,VNST,IsOnlineSale,Model,Doors,SubModel,PurchYearMonth,MMRAcquisitionAuctionAveragePriceBin,VehBCostBin,VehOdoBin,WarrantyCostBin
0,0,MANHEIM,2003,7,CHEVROLET,V6,150,AUTO,1.0,AMERICAN,...,AZ,0,AVALANCHE,4D,SUV-PICKUP,2010-10,"[6708.6, 8944.8)","[10629.8, 13533.067)","[71652.6, 78646.667)","[1309.6, 1733.4)"
1,0,OTHER,2006,4,FORD,NaN,BAS,MANUAL,1.0,AMERICAN,...,NC,0,MUSTANG,2D,COUPE,2010-01,"[6708.6, 8944.8)","[4823.267, 7726.533)","[71652.6, 78646.667)","[462.0, 885.8)"
2,0,MANHEIM,2004,5,DODGE,NaN,ES,AUTO,1.0,AMERICAN,...,AZ,0,STRATUS,4D,SEDAN,2009-09,"[2236.2, 4472.4)","[4823.267, 7726.533)","[71652.6, 78646.667)","[1309.6, 1733.4)"
3,0,OTHER,2004,6,HYUNDAI,NaN,BAS,MANUAL,1.0,NOT AMERICAN,...,CO,0,TIBURON,2D,COUPE,2010-01,"[2236.2, 4472.4)","[4823.267, 7726.533)","[71652.6, 78646.667)","[885.8, 1309.6)"
4,0,MANHEIM,2004,5,CHEVROLET,6C,NOR,AUTO,1.0,AMERICAN,...,CO,0,TRAILBLAZER,4D,SUV,2009-03,"[4472.4, 6708.6)","[4823.267, 7726.533)","[71652.6, 78646.667)","[885.8, 1309.6)"
5,1,OTHER,2004,6,FORD,4.0L,XLS,AUTO,1.0,AMERICAN,...,SC,0,EXPLORER,4D,SUV,2010-12,"[4472.4, 6708.6)","[4823.267, 7726.533)","[78646.667, 85640.733)","[885.8, 1309.6)"
6,0,OTHER,2008,2,CHEVROLET,3.5L,LS,AUTO,2.0,AMERICAN,...,TX,0,IMPALA,4D,SEDAN,2010-01,"[6708.6, 8944.8)","[7726.533, 10629.8)","[85640.733, 92634.8)","[1733.4, 2157.2)"
7,0,ADESA,2002,7,CHRYSLER,2.4L,LIM,AUTO,1.0,AMERICAN,...,TN,0,PT CRUISER,4D,SEDAN,2009-07,"[2236.2, 4472.4)","[4823.267, 7726.533)","[78646.667, 85640.733)","[1733.4, 2157.2)"
8,0,OTHER,2006,4,DODGE,3.7L,SXT,AUTO,1.0,AMERICAN,...,SC,0,DURANGO,4D,SUV,2010-06,"[8944.8, 11181.0)","[7726.533, 10629.8)","[71652.6, 78646.667)","[885.8, 1309.6)"
9,0,MANHEIM,2002,7,BUICK,3.1L,CUS,AUTO,2.0,AMERICAN,...,FL,0,CENTURY,4D,SEDAN,2009-03,"[2236.2, 4472.4)","[4823.267, 7726.533)","[64658.533, 71652.6)","[1733.4, 2157.2)"


** Append dell'Intestazione delle Colonne **

In [38]:
for col in df_Test_Association.columns:
    df_Test_Association[col] = df_Test_Association[col].astype(str) + '_' + str(col)

In [39]:
df_Test_Association

,IsBadBuy,Auction,VehYear,VehicleAge,Make,Engine,Trim,Transmission,WheelTypeID,Nationality,...,VNST,IsOnlineSale,Model,Doors,SubModel,PurchYearMonth,MMRAcquisitionAuctionAveragePriceBin,VehBCostBin,VehOdoBin,WarrantyCostBin
0,0_IsBadBuy,MANHEIM_Auction,2003_VehYear,7_VehicleAge,CHEVROLET_Make,V6_Engine,150_Trim,AUTO_Transmission,1.0_WheelTypeID,AMERICAN_Nationality,...,AZ_VNST,0_IsOnlineSale,AVALANCHE_Model,4D_Doors,SUV-PICKUP_SubModel,2010-10_PurchYearMonth,"[6708.6, 8944.8)_MMRAcquisitionAuctionAverageP...","[10629.8, 13533.067)_VehBCostBin","[71652.6, 78646.667)_VehOdoBin","[1309.6, 1733.4)_WarrantyCostBin"
1,0_IsBadBuy,OTHER_Auction,2006_VehYear,4_VehicleAge,FORD_Make,nan_Engine,BAS_Trim,MANUAL_Transmission,1.0_WheelTypeID,AMERICAN_Nationality,...,NC_VNST,0_IsOnlineSale,MUSTANG_Model,2D_Doors,COUPE_SubModel,2010-01_PurchYearMonth,"[6708.6, 8944.8)_MMRAcquisitionAuctionAverageP...","[4823.267, 7726.533)_VehBCostBin","[71652.6, 78646.667)_VehOdoBin","[462.0, 885.8)_WarrantyCostBin"
2,0_IsBadBuy,MANHEIM_Auction,2004_VehYear,5_VehicleAge,DODGE_Make,nan_Engine,ES_Trim,AUTO_Transmission,1.0_WheelTypeID,AMERICAN_Nationality,...,AZ_VNST,0_IsOnlineSale,STRATUS_Model,4D_Doors,SEDAN_SubModel,2009-09_PurchYearMonth,"[2236.2, 4472.4)_MMRAcquisitionAuctionAverageP...","[4823.267, 7726.533)_VehBCostBin","[71652.6, 78646.667)_VehOdoBin","[1309.6, 1733.4)_WarrantyCostBin"
3,0_IsBadBuy,OTHER_Auction,2004_VehYear,6_VehicleAge,HYUNDAI_Make,nan_Engine,BAS_Trim,MANUAL_Transmission,1.0_WheelTypeID,NOT AMERICAN_Nationality,...,CO_VNST,0_IsOnlineSale,TIBURON_Model,2D_Doors,COUPE_SubModel,2010-01_PurchYearMonth,"[2236.2, 4472.4)_MMRAcquisitionAuctionAverageP...","[4823.267, 7726.533)_VehBCostBin","[71652.6, 78646.667)_VehOdoBin","[885.8, 1309.6)_WarrantyCostBin"
4,0_IsBadBuy,MANHEIM_Auction,2004_VehYear,5_VehicleAge,CHEVROLET_Make,6C_Engine,NOR_Trim,AUTO_Transmission,1.0_WheelTypeID,AMERICAN_Nationality,...,CO_VNST,0_IsOnlineSale,TRAILBLAZER_Model,4D_Doors,SUV_SubModel,2009-03_PurchYearMonth,"[4472.4, 6708.6)_MMRAcquisitionAuctionAverageP...","[4823.267, 7726.533)_VehBCostBin","[71652.6, 78646.667)_VehOdoBin","[885.8, 1309.6)_WarrantyCostBin"
5,1_IsBadBuy,OTHER_Auction,2004_VehYear,6_VehicleAge,FORD_Make,4.0L_Engine,XLS_Trim,AUTO_Transmission,1.0_WheelTypeID,AMERICAN_Nationality,...,SC_VNST,0_IsOnlineSale,EXPLORER_Model,4D_Doors,SUV_SubModel,2010-12_PurchYearMonth,"[4472.4, 6708.6)_MMRAcquisitionAuctionAverageP...","[4823.267, 7726.533)_VehBCostBin","[78646.667, 85640.733)_VehOdoBin","[885.8, 1309.6)_WarrantyCostBin"
6,0_IsBadBuy,OTHER_Auction,2008_VehYear,2_VehicleAge,CHEVROLET_Make,3.5L_Engine,LS_Trim,AUTO_Transmission,2.0_WheelTypeID,AMERICAN_Nationality,...,TX_VNST,0_IsOnlineSale,IMPALA_Model,4D_Doors,SEDAN_SubModel,2010-01_PurchYearMonth,"[6708.6, 8944.8)_MMRAcquisitionAuctionAverageP...","[7726.533, 10629.8)_VehBCostBin","[85640.733, 92634.8)_VehOdoBin","[1733.4, 2157.2)_WarrantyCostBin"
7,0_IsBadBuy,ADESA_Auction,2002_VehYear,7_VehicleAge,CHRYSLER_Make,2.4L_Engine,LIM_Trim,AUTO_Transmission,1.0_WheelTypeID,AMERICAN_Nationality,...,TN_VNST,0_IsOnlineSale,PT CRUISER_Model,4D_Doors,SEDAN_SubModel,2009-07_PurchYearMonth,"[2236.2, 4472.4)_MMRAcquisitionAuctionAverageP...","[4823.267, 7726.533)_VehBCostBin","[78646.667, 85640.733)_VehOdoBin","[1733.4, 2157.2)_WarrantyCostBin"
8,0_IsBadBuy,OTHER_Auction,2006_VehYear,4_VehicleAge,DODGE_Make,3.7L_Engine,SXT_Trim,AUTO_Transmission,1.0_WheelTypeID,AMERICAN_Nationality,...,SC_VNST,0_IsOnlineSale,DURANGO_Model,4D_Doors,SUV_SubModel,2010-06_PurchYearMonth,"[8944.8, 11181.0)_MMRAcquisitionAuctionAverage...","[7726.533, 10629.8)_VehBCostBin","[71652.6, 78646.667)_VehOdoBin","[885.8, 1309.6)_WarrantyCostBin"
9,0_IsBadBuy,MANHEIM_Auction,2002_VehYear,7_VehicleAge,BUICK_Make,3.1L_Engine,CUS_Trim,AUTO_Transmission,2.0_WheelTypeID,AMERICAN_Nationality,...,FL_VNST,0_IsOnlineSale,CENTURY_Model,4D_Doors,SEDAN_SubModel,2009-03_PurchYearMonth,"[2236.2, 4472.4)_MMRAcquisitionAuctionAverageP...","[4823.267, 7726.533)_V

In [40]:
df_Test_Association.nunique()

IsBadBuy                                  2
Auction                                   3
VehYear                                   9
VehicleAge                               10
Make                                     31
Engine                                  111
Trim                                    116
Transmission                              3
WheelTypeID                               5
Nationality                               3
Size                                     13
TopThreeAmericanName                      5
BYRNO                                    70
VNZIP1                                  142
VNST                                     37
IsOnlineSale                              2
Model                                   223
Doors                                     5
SubModel                                 32
PurchYearMonth                           24
MMRAcquisitionAuctionAveragePriceBin     11
VehBCostBin                               8
VehOdoBin                       

## 3) Tasformazioni per la Classification

# Files in Output per le analisi successive

** - File per il Clustering **

In [48]:
df_Clustering.to_csv(path_or_buf='Training_DU_Clustering.csv' ,index=False)

In [49]:
df_Clustering_forResultsAnalysis.to_csv(path_or_buf='Training_DU_Clustering_forResultsAnalysis.csv' ,index=False)

** - File per la Association Rule **

In [42]:
df_Association.to_csv(path_or_buf='Training_DU_Association.csv' ,index=False)

In [41]:
df_Test_Association.to_csv(path_or_buf='Test_DU_Association.csv' ,index=False)

** - File per la Classification **

In [50]:
#df_Classification.to_csv(path_or_buf='Training_DU_Classification.csv' ,index=False)